In [1]:
import pandas as pd
customers = pd.read_csv(r'..\data\processed\customers_cleaned.csv')
transactions = pd.read_csv(r'..\data\processed\transactions_cleaned.csv')   

In [2]:
transactions['visit_date'] = pd.to_datetime(transactions['visit_date'])
customers['registration_date'] = pd.to_datetime(customers['registration_date'])

In [3]:
reference_date = transactions['visit_date'].max() + pd.Timedelta(days=1)

In [4]:
total_transactions = transactions.groupby('customer_id').size().reset_index(name='total_transactions')

In [5]:
total_spending = transactions.groupby('customer_id')['total_spending'].agg(
total_spending = 'sum',
avg_spending = 'mean',).reset_index()

In [6]:
last_visit = transactions.groupby('customer_id')['visit_date'].max().reset_index()
last_visit['recency_days'] = (reference_date - last_visit['visit_date']).dt.days
last_visit = last_visit[['customer_id', 'recency_days']]

In [7]:
transactions_features = (total_transactions
                        .merge(total_spending, on='customer_id')
                        .merge(last_visit, on='customer_id'))

In [8]:
customers['customer_tenure_days'] = (reference_date - customers['registration_date']).dt.days

In [9]:
customers_encoded = pd.get_dummies(customers[['customer_id', 'gender', 'city', 'customer_tenure_days']], 
                                   columns=['gender', 'city'], drop_first=True)

In [10]:
customers_features = customers_encoded.merge(transactions_features, on='customer_id', how='left')

In [11]:
customers_features.fillna({
    'total_transactions': 0,
    'total_spending': 0,
    'avg_spending': 0,
    'recency_days': customers_features['recency_days'].max()
})

,customer_id,customer_tenure_days,gender_M,city_Jakarta,city_Malang,city_Solo,city_Surabaya,city_Yogyakarta,total_transactions,total_spending,avg_spending,recency_days
0,1001,768,True,False,False,True,False,False,16,4352665,272041.562500,155
1,1002,590,False,False,False,True,False,False,13,2753160,211781.538462,16
2,1003,1029,True,True,False,False,False,False,18,4810529,267251.611111,9
3,1004,552,True,False,False,False,True,False,12,3566812,297234.333333,14
4,1005,1045,True,False,False,False,False,False,15,4548499,303233.266667,67
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1496,432,True,False,False,True,False,False,14,4258704,304193.142857,50
496,1497,656,True,False,False,True,False,False,15,3937832,262522.133333,39
497,1498,1069,False,False,True,False,False,False,14,3829355,273525.357143,8
498,1499,989,True,False,False,False,False,False,19,3924316,206542.947368,26


In [12]:
CHURN_THRESHOLD_DAYS = 90
customers_features['is_churned'] = (customers_features['recency_days'] > CHURN_THRESHOLD_DAYS).astype(int)

print(customers_features['is_churned'].value_counts())


is_churned
0    421
1     79
Name: count, dtype: int64


In [13]:
customers_features = customers_features.drop(columns=['recency_days'])

In [14]:
customers_features.to_csv(r'..\data\processed\customers_features.csv', index=False)